In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '/data/programs/probe_position_correction_w_ptychonn/pppc')
sys.path.insert(0, '/data/programs/probe_position_correction_w_ptychonn')
import numpy as np
import torch
import matplotlib.pyplot as plt

import pppc
from pppc.configs import PtychoNNTrainingConfigDict
from pppc.ptychonn.trainer import PtychoNNTrainer, PtychoNNHyperparameterScanner
from pppc.ptychonn.dataset_handle import HDF5Dataset
from pppc.ptychonn.model import PtychoNNModel, PtychoNNPhaseOnlyModel, PtychoNNTransposedConvModel

[2023-12-15 16:53:53,342] Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2023-12-15 16:53:53,345] NumExpr defaulting to 8 threads.


Create a baseline config dict. 

In [3]:
base_config_dict = PtychoNNTrainingConfigDict()
base_config_dict['batch_size_per_process'] = 64
base_config_dict['num_epochs'] = 60
base_config_dict['learning_rate_per_process'] = 1e-3
base_config_dict['optimizer'] = 'adam'
base_config_dict['model_save_dir'] = '../../trained_models/model_36SpiralDatasets_compWithTransConv'
base_config_dict['validation_ratio'] = 0.01
dataset = HDF5Dataset('data/data_train.h5', verbose=False)
base_config_dict['dataset'] = dataset
base_config_dict['model'] = None
base_config_dict['debug'] = False

In [4]:
param_scan_dict = {
    'model': [(PtychoNNTransposedConvModel, {'n_levels': 5}), (PtychoNNTransposedConvModel, {'n_levels': 4}), (PtychoNNModel, {'n_levels': 4}), (PtychoNNModel, {'n_levels': 5})],
    'loss_function': [torch.nn.MSELoss, torch.nn.L1Loss],
    'batch_size_per_process': [32, 64],
    'learning_rate_per_process': [1e-3, 1e-4]
}

In [5]:
scanner = PtychoNNHyperparameterScanner(param_scan_dict, base_config_dict)
scanner.build()
scanner.keep_models_in_memory = True

In [6]:
scanner.result_table

,model,loss_function,batch_size_per_process,learning_rate_per_process,min_val_loss_total,min_val_loss_mag,min_val_loss_phase,epoch_min_val_loss
0,PtychoNNTransposedConvModel_nLevels_5,<class 'torch.nn.modules.loss.MSELoss'>,32,0.0010,0.0,0.0,0.0,0.0
1,PtychoNNTransposedConvModel_nLevels_5,<class 'torch.nn.modules.loss.MSELoss'>,32,0.0001,0.0,0.0,0.0,0.0
2,PtychoNNTransposedConvModel_nLevels_5,<class 'torch.nn.modules.loss.MSELoss'>,64,0.0010,0.0,0.0,0.0,0.0
3,PtychoNNTransposedConvModel_nLevels_5,<class 'torch.nn.modules.loss.MSELoss'>,64,0.0001,0.0,0.0,0.0,0.0
4,PtychoNNTransposedConvModel_nLevels_5,<class 'torch.nn.modules.loss.L1Loss'>,32,0.0010,0.0,0.0,0.0,0.0
5,PtychoNNTransposedConvModel_nLevels_5,<class 'torch.nn.modules.loss.L1Loss'>,32,0.0001,0.0,0.0,0.0,0.0
6,PtychoNNTransposedConvModel_nLevels_5,<class 'torch.nn.modules.loss.L1Loss'>,64,0.0010,0.0,0.0,0.0,0.0
7,PtychoNNTransposedConvModel_nLevels_5,<class 'torch.nn.modules.loss.L1Loss'>,64,0.0001,0.0,0.0,0.0,0.0
8,PtychoNNTransposedConvModel_nLevels_4,<class 'torch.nn.modules.loss.MSELoss'>,32,0.0010,0.0,0.0,0.0,0.0
9,PtychoNNTransposedConvModel_nLevels_4,<class 'torch.nn.modules.loss.MSELoss'>,32,0.0001,0.0,0.0,0.0,0.0


In [ ]:
scanner.run()
scanner.result_table

  0%|                                                                                                                                                                                                                                                    | 0/32 [00:00<?, ?it/s][2023-12-15 16:53:55,651] Using DataParallel with 2 devices.


In [ ]:
scanner.plot_all_training_history()

In [ ]:
scanner.run_testing_for_all((0, 1))